Homework # 4 CompSci X 433.6

In [406]:
from os.path import join, dirname, abspath
import xlrd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
import pandas as pd
import openpyxl as pxl

# Excel Write Function

In [407]:
def writeExcelData(x, excelfile, sheetname, startrow, startcol):
    from pandas import DataFrame, ExcelWriter
    from openpyxl import load_workbook
    df=DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname, startrow=startrow-1, startcol=startcol-1, header=False, index=False)
    writer.save()
    writer.close()

In [408]:
excelfile=r"Assignment_4_Data_and_Template.xlsx"

# Utility Functions

In [409]:
def kesler(a):
    tempa=MultiLabelBinarizer().fit_transform(a)
    for i in range(tempa.shape[0]):
        for j in range(tempa.shape[1]):
            if (tempa[i][j]==0):
                tempa[i][j]=-1
    return tempa

In [410]:
def OneHot1D(a):
    tempa=np.zeros((a.shape[0]), dtype=np.int)
    print(tempa.shape)
    for i in range(a.shape[0]):
        if (a[i]< 0):
            tempa[i]=-1
        else:
            tempa[i]=1
    return tempa

In [411]:
def OneHot2D(a):
    t=np.full((a.shape[0],a.shape[1]), -1, dtype=int)
    maxindex=np.argmax(a, axis=1)
    for i in range(a.shape[0]):
        t[i][maxindex[i]]=1
    return t

# Training Data

In [412]:
xl = pd.ExcelFile("Assignment_4_Data_and_Template.xlsx")
#df = xl.parse(xl.sheet_names[0],header=0, index_col=0)
df = xl.parse(xl.sheet_names[0])
df.head()


,Temperature,x-acc,y-acc,z-acc,Pressure,Load,Flow,Unnamed: 7,Unnamed: 8,Nitrogen,Unnamed: 10,Unnamed: 11,Frequency,Unnamed: 13,Unnamed: 14,Failure,Type
0,149,118,136,113,113,105,0,0,1,0,0,1,0,0,1,-1,4
1,171,218,83,265,222,129,0,1,0,0,1,0,0,1,0,1,0
2,295,267,177,160,317,216,0,1,0,1,0,0,0,1,0,1,2
3,102,155,147,266,197,119,0,1,0,0,1,0,1,0,0,-1,3
4,129,143,169,134,131,101,0,0,1,0,0,1,0,0,1,-1,4


In [413]:
A1 = df.reset_index().values
A1T=np.transpose(A1)
X = np.transpose(A1T[1:16])
B = np.ones((X.shape[0],1), dtype=np.int)
XA = np.concatenate((B,X), axis=1)
TFail = np.transpose(A1T[16:17])
TType = np.transpose(A1T[17:18])
#print(X.shape)
#print(X[:16])
print(XA.shape)
#print(XA)
#print(TFail)
print(TFail.shape)
#print(TType)
print(TType.shape)

(6600, 16)
(6600, 1)
(6600, 1)


In [414]:
temp = np.transpose(TFail)
# ground truth for failures
TFailT=temp[0]
print(TFailT.shape)


(6600,)


In [415]:
temp = np.transpose(TType)
# ground truth for types
TTypeT=temp[0]
print(TTypeT.shape)

(6600,)


In [416]:
temp = np.transpose(TFail)
TFailT=temp[0]
NumTrainNoFail=(TFailT == 1).sum()
NumTrainFail=(TFailT == -1).sum()
NumTrain=len(TFailT)
print("NumTrain = {}, Failures = {}, Non Failures = {}".format(NumTrain,NumTrainFail,NumTrainNoFail))

NumTrain = 6600, Failures = 3300, Non Failures = 3300


In [417]:
TTypeK=kesler(TType)
print(TTypeK)

[[-1 -1 -1 -1  1 -1]
 [ 1 -1 -1 -1 -1 -1]
 [-1 -1  1 -1 -1 -1]
 ..., 
 [ 1 -1 -1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1]
 [-1 -1 -1  1 -1 -1]]


# Training

In [418]:
XaInv = np.linalg.pinv(XA)
WFail = np.dot(XaInv, TFail)
print(WFail.shape)
print(WFail)


(16, 1)
[[  2.94413221e-01]
 [ -7.32266635e-04]
 [ -1.08866787e-03]
 [ -5.54479443e-04]
 [ -6.23305624e-04]
 [ -2.58846750e-04]
 [ -8.87480321e-04]
 [  6.32307013e-02]
 [  1.86371969e-01]
 [  4.48105509e-02]
 [  5.27055537e-01]
 [ -1.92463071e-01]
 [ -4.01792449e-02]
 [  1.65623856e-01]
 [  9.07088949e-01]
 [ -7.78299584e-01]]


In [419]:
WType = np.dot(XaInv, TTypeK)
print(WType.shape)
print(WType)


(16, 6)
[[  3.10076536e-01   2.06367231e-01  -1.22203055e+00  -1.02470911e-01
    7.95346173e-02  -1.27147693e+00]
 [ -1.09947231e-03  -1.34378797e-03   1.71099365e-03  -7.36257903e-04
   -7.07445591e-05   1.53926910e-03]
 [ -1.50009500e-03  -1.87036044e-03   2.28178757e-03  -4.84292730e-04
   -1.66856669e-04   1.73981727e-03]
 [ -9.56771519e-04  -5.17947549e-04   9.20239625e-04  -7.15249651e-05
   -5.12823616e-05   6.77286770e-04]
 [ -1.95908802e-03  -1.70154762e-03   3.03733002e-03  -1.57517925e-03
   -1.56328122e-04   2.35481300e-03]
 [ -8.82855798e-04  -7.66170364e-04   1.39017941e-03  -7.19173484e-04
   -1.25349559e-04   1.10336979e-03]
 [ -1.31128948e-03  -1.02932482e-03   1.45313398e-03   1.42700183e-04
   -6.00243272e-05   8.04804465e-04]
 [  5.47900875e-02   1.40094335e-02  -3.38902153e-01  -4.20322099e-02
    4.13375972e-01  -7.67907797e-01]
 [  2.58375804e-01   2.26702639e-01  -6.32039808e-01   3.48027685e-02
   -7.35133390e-01   1.80625319e-01]
 [ -3.08935563e-03  -3.434484

In [420]:
print(XA.shape)
print(WFail.shape)
TTrainF = np.dot(XA, WFail)
TTrainFOH = OneHot1D(TTrainF)
print(TTrainFOH.shape)
print(TTrainFOH)

(6600, 16)
(16, 1)
(6600,)
(6600,)
[-1  1  1 ...,  1  1 -1]


In [421]:
def ConfMat(t,p):
    c=np.zeros((2,2), dtype=np.int)
    for i in range(t.shape[0]):
        if ((t[i]==-1) and (p[i]==-1)):
            c[0,0]+=1
        if ((t[i]==-1) and (p[i]==1)):
            c[0,1]+=1
        if ((t[i]==1) and (p[i]==-1)):
            c[1,0]+=1
        if ((t[i]==1) and (p[i]==1)):
            c[1,1]+=1
    return c

In [422]:
# CMTrainFail = confusion_matrix(y_true, y_pred)
CMF = confusion_matrix(TFailT, TTrainFOH)
CMF2 = ConfMat(TFailT, TTrainFOH)
print(CMF)
print(CMF2)

[[3229   71]
 [1150 2150]]
[[3229   71]
 [1150 2150]]


In [423]:
Accuracy = (CMF[0,0]+CMF[1,1])/(CMF[0,0]+CMF[0,1]+CMF[1,0]+CMF[1,1])
Sensitivity = (CMF[1,1])/(CMF[1,1]+CMF[1,0])
Specificity = (CMF[0,0])/(CMF[0,0]+CMF[0,1])
PPV = (CMF[1,1])/(CMF[0,1]+CMF[1,1])

# Training Data : Type Classification

In [424]:
print(XA.shape)
print(WType.shape)
TTrainType = np.dot(XA, WType)
maxTrindex=np.argmax(TTrainType, axis=1)
print(maxTrindex)

(6600, 16)
(16, 6)
[4 0 2 ..., 2 0 5]


In [425]:
# CMTrainFail = confusion_matrix(y_true, y_pred)
CMTrainType = confusion_matrix(TTypeT, maxTrindex)
print(CMTrainType)

[[ 652   11  390    1    0   46]
 [   0  739    0  270    0   91]
 [ 216    0  872    0    0   12]
 [  15  532    2  176    0  375]
 [   0    7    0    0 1090    3]
 [  30  256    9   76    4  725]]


In [426]:
def ppvcomp(c):
    colsum = np.sum(c, axis=0)
    print(colsum)
    p=np.zeros((c.shape[1]))   
    r=np.zeros((2,2))   
    for i in range(c.shape[1]):
        p[i] = c[i][i]/colsum[i]
    maxindex=np.argmax(p)
    minindex=np.argmin(p)
    hppv=100.0*p[maxindex]
    lppv=100.0*p[minindex]
    return hppv,maxindex,lppv,minindex
       

In [427]:
hppv,ppvmaxi,lppv,ppvmini=ppvcomp(CMTrainType)
print(hppv,ppvmaxi,lppv,ppvmini)

[ 913 1545 1273  523 1094 1252]
99.634369287 4 33.6520076482 3


In [428]:
np.sum([[0, 1, 3], [0, 5, 4]], axis=0)

array([0, 6, 7])

In [429]:
np.sum([[0, 1, 3], [0, 5, 4]], axis=1)

array([4, 9])

# Testing - Classification

In [430]:
dft = xl.parse(xl.sheet_names[2])
dft.head()


,Student to fill in boxes shaded,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Temperature,x-acc,y-acc,z-acc,Pressure,Load,Flow,NaN,NaN,Nitrogen,NaN,NaN,Frequency,NaN,NaN,Failure,Type
3,160,200,140,140,260,140,0,1.0,0.0,0,1.0,0.0,1,0.0,0.0,NaN,NaN
4,140,160,120,160,140,160,0,0.0,1.0,0,0.0,1.0,0,0.0,1.0,NaN,NaN


# Test Data

In [431]:
AT1 = dft.reset_index(3).values
AT1=AT1[3:]
AT1T=np.transpose(AT1)
XT = np.transpose(AT1T[1:16])
B = np.ones((XT.shape[0],1), dtype=np.int)
XTA = np.concatenate((B,XT), axis=1)


# Test Data : Failure Classification

In [432]:
NumTest = XTA.shape[0]
print(XTA.shape)
print(WFail.shape)
TTestF = np.dot(XTA, WFail)
TTestFOH = OneHot1D(TTestF)
print(TTestFOH.shape)
print(TTestFOH)


(50, 16)
(16, 1)
(50,)
(50,)
[-1 -1  1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1
 -1  1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1  1 -1  1 -1 -1 -1 -1  1 -1  1 -1  1]


# Test Data : Type Classification

In [433]:
print(XTA.shape)
print(WType.shape)
TTestType = np.dot(XTA, WType)
TTestTypeClass=np.argmax(TTestType, axis=1)
print(TTestTypeClass)

(50, 16)
(16, 6)
[1 4 0 4 4 5 0 4 5 1 2 3 1 4 0 1 5 5 1 0 5 5 4 5 4 3 2 4 4 4 2 0 2 2 5 1 1
 5 2 5 2 4 1 5 1 2 5 2 4 2]


# Writing to Excel

In [434]:
def writex(val,row,col,sheet):
    arr=np.array([[val]])
    writeExcelData(arr[0:1,:], excelfile, sheet, row, col)

In [435]:
# Classifiers Page 

# Binary Classifier
writeExcelData(WFail, excelfile, 'Classifiers', 5, 1)

# 6-Class Classifier
writeExcelData(WType[:,:], excelfile, 'Classifiers', 5, 5)

print("Done Classifier")

# To be classified Page 

# Failure Classification: Test Data
writeExcelData(TTestFOH, excelfile, 'To be classified', 5, 16)

# Type Classification: Test Data
writeExcelData(TTestTypeClass, excelfile, 'To be classified', 5, 17)

print("Done To be classified")


# Performance Page 

# Binary Classifier Confusion Matrix
writeExcelData(CMF[:,:], excelfile, 'Performance', 10, 3)

# 6-Class Classifier
writeExcelData(CMTrainType[:,:], excelfile, 'Performance', 19, 3)

# Accuracy, Sensitivity, Specificity, PPV
writex(Accuracy,8,7,'Performance')
writex(Sensitivity,9,7,'Performance')
writex(Specificity,10,7,'Performance')
writex(PPV,11,7,'Performance')

# Highest and Lowest PPV
writex(hppv,20,12,'Performance')
writex(ppvmaxi,20,13,'Performance')
writex(lppv,21,12,'Performance')
writex(ppvmini,21,13,'Performance')

print("Done Performance")


Done Classifier
Done To be classified
Done Performance
